## ML lab 04-2: TensorFlow로 파일에서 데이터 읽어오기 (new)

#### NumPy

In [1]:
# data load

import numpy as np
xy = np.loadtxt('data-01-test-score.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0 : -1] # 모든행, 0에서 마지막 끝행은 빼고 가져옴
y_data = xy[:, [-1]] # 모든행, 마지막 끝행만 가져옴

print(x_data.shape, len(x_data))
print(y_data.shape, len(y_data))

(6, 3) 6
(6, 1) 6


In [2]:
# list 형태

import tensorflow as tf
import matplotlib.pyplot as plt
tf.set_random_seed(777)

xy = np.loadtxt('data-01-test-score.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0 : -1] # 모든행, 0에서 마지막 끝행은 빼고 가져옴
y_data = xy[:, [-1]] # 모든행, 마지막 끝행만 가져옴

# placeholder
X = tf.placeholder(tf.float32, shape = [None, 3])
Y = tf.placeholder(tf.float32, shape = [None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

# hypothesis
hypothesis = tf.matmul(X, W) + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

# launch
sess = tf.Session()

# initial
sess.run(tf.global_variables_initializer())

for step in range(2000+1) :
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                    feed_dict = {X : x_data, Y : y_data})
    if step % 500 == 0 :
        print(step, 'cost :', cost_val, '\nPrediction:', hy_val)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 cost : 20344.273 
Prediction: [[22.048063 ]
 [21.619787 ]
 [24.096693 ]
 [22.293005 ]
 [18.633902 ]
 [ 7.2669735]]
500 cost : 5.194362 
Prediction: [[155.39178]
 [182.79918]
 [182.4304 ]
 [194.83575]
 [141.69955]
 [ 98.26765]]
1000 cost : 4.1113534 
Prediction: [[154.95578]
 [183.00641]
 [182.2495 ]
 [194.73842]
 [141.95724]
 [ 98.69708]]
1500 cost : 3.327547 
Prediction: [[154.58617]
 [183.18124]
 [182.09547]
 [194.65872]
 [142.1719 ]
 [ 99.06362]]
2000 cost : 2.7586195 
Prediction: [[154.27278 ]
 [183.32866 ]
 [181.96426 ]
 [194.5939  ]
 [142.35019 ]
 [ 99.376816]]


In [ ]:
# my score
print('Your final score will be :', sess.run(hypothesis, 
       feed_dict = {X : [[100, 70, 101]]}))

Your final score will be : [[171.15341]]


#### tf.train.batch

In [ ]:
# collection batches (batch는 펌프와 같은 역할)
# train_x_batch, train_y_batch = \
# tf.train.batch([xy[0:-1], xy[-1:]], batch_size = 10)

# sess = tf.Session()

# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(sess = sess, coord = coord)

# for step in range(2000+1) :
#     x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
#     ...
# coord.request_stop()
# coord.join(threads)

In [ ]:
import tensorflow as tf

# read file
filename_queue = tf.train.string_input_producer(
['data-01-test-score.csv'], shuffle = False, name = 'filename_queue')

# read 정의
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# decode result
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults = record_defaults)

# collection batches
train_x_batch, train_y_batch = \
tf.train.batch([xy[0:-1], xy[-1:]], batch_size = 10)

# placeholder
X = tf.placeholder(tf.float32, shape = [None, 3])
Y = tf.placeholder(tf.float32, shape = [None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

# hypothesis
hypothesis = tf.matmul(X, W) + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

# launch
sess = tf.Session()

# initial
sess.run(tf.global_variables_initializer())

for step in range(2000+1) :
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                          feed_dict = {X : x_batch, Y : y_batch})
    
    if step % 500 == 0 :
        print(step, 'cost :', cost_val, '\nPrediction:', hy_val)
        
coord.request_stop()
coord.join(threads)